<a id="top"></a>

# Db2 11.5 SQL Enhancements
Updated: 2019-10-03

### SQL Enhancements
Moving from one database vendor to another can sometimes be difficult due to syntax differences between data types, functions, and language elements. Db2 already has a high degree of compatibility with Oracle PLSQL along with some of the Oracle data types. 

Db2 11.5 introduces some additional data type and function compatibility that will reduce some of the migration effort required when porting from other systems. 

### Load Db2 Extensions and Connect to the Database
The `connection` notebook contains the `CONNECT` statement which allows access to the `SAMPLE` database. If you need to modify the connection information, edit the `connection.ipynb` notebook.

In [ ]:
%run ../db2.ipynb
%run ../connection.ipynb

We populate the database with the EMPLOYEE and DEPARTMENT tables so that we can run the various examples.

In [ ]:
%sql -sampledata

# Table of Contents

* [If NOT EXISTS](#ifnotexists)

* [If EXISTS](#ifexists)

* [Columnar Data Types](#blobclob)

* [Quote Identifiers and Literals](#quotes)

* [Oracle Compatibility Libraries](#oracle)

<a id='ifnotexists'></a>

## IF NOT EXISTS

The `IF NOT EXISTS` clause is used during table creation. This SQL specifies that no error message is shown when the table cannot be created because a table with the specified name already exists in the current database and schema. Note that the existing table and the specified table in the command are not compared so the application must ensure that the target table and rows are as expected.
```sql
CREATE TABLE IF NOT EXISTS CUSTOMER …
```

Use this option for scripted applications that are running SQL commands. This syntax will suppress the **Table already exists** error message, so the scripted application is not impacted or halted.

You cannot use the `IF NOT EXISTS` option with the `AS SELECT` clause. 

Unless other errors prevent the creation of the table, a `CREATE TABLE` message is returned although no table is created. An error code is ignored if a table with the specified name already exists.

This SQL will delete an existing table and then re-create it.

In [ ]:
%%sql 
DROP TABLE DEMO_TABLE;
CREATE TABLE DEMO_TABLE( SOMEVALUE VARCHAR(100) );

A normal `CREATE TABLE` statement will fail with an error since the table already exists.

In [ ]:
%sql CREATE TABLE DEMO_TABLE( SOMEVALUE VARCHAR(100) );

Using the `IF NOT EXISTS` clause will ignore the fact that the table already exists and complete successfully.

In [ ]:
%sql CREATE TABLE IF NOT EXISTS DEMO_TABLE( SOMEVALUE VARCHAR(100) );

[Back to Top](#top)
<a id='ifexists'></a>

## IF EXISTS
 
The `IF EXISTS` is used when dropping a table and specifies that no error message is shown when the table is not dropped because it does not currently exist in the database. 
```sql
DROP TABLE CUSTOMER IF EXISTS
```
Use this option for scripted applications that are running SQL commands. This option will suppress the **Table not found** error message so the scripted application is not impacted or halted.

The `DROP TABLE` message is returned although no table was dropped. Failure is ignored if a table with the specified name does not exist.


First we create a table (**Note:** The Db2 magic commands ignore errors!)

In [ ]:
%%sql -q
DROP TABLE DEMO_TABLE;
CREATE TABLE DEMO_TABLE( SOMEVALUE VARCHAR(100) );

This first `DROP` statement will drop the table without an error message.

In [ ]:
%%sql
DROP TABLE DEMO_TABLE;

If we try dropping it again using the normal syntax, we will get an error message.

In [ ]:
%%sql
DROP TABLE DEMO_TABLE;

Adding the `IF EXISTS` clause to the `DROP` statement will ignore the error message.

In [ ]:
%%sql
DROP TABLE IF EXISTS DEMO_TABLE;

[Back to Top](#top)
<a id='blobclob'></a>

## Columnar Data Types

Db2 11.5 adds two new data types to columnar tables: CLOB and BLOB. These two data types can contain any combination 
of characters (CLOB) or binary values (BLOB). BLOBs are not affected by the codepage of the server that the values are stored on.

A CLOB or BLOB data type can have a maximum length of 2 Gb.

This example shows the use of these two data types in a columnar table.

In [ ]:
%%sql
DROP TABLE CLOBBLOB;

CREATE TABLE CLOBBLOB
  (
  AUDIO_TITLE CLOB(1024),
  AUDIO_TRACK BLOB(1 M)
  ) ORGANIZE BY COLUMN;

When using a CLOB or BLOB object, an `INLINE LENGTH` specification should be used to try and place as much of the data on the data page to take advantage of the performance advantage provided by the buffer pool caching effect. If you do not specify an inline length for large objects, the data will not reside in the buffer pool and searching and retrieval of this data will take an additional I/O operation.
The following SQL will recreate the table specifying an inline length for the columns.

In [ ]:
%%sql
DROP TABLE CLOBBLOB;

CREATE TABLE CLOBBLOB
  (
  AUDIO_TITLE CLOB(1024) INLINE LENGTH 1024,
  AUDIO_TRACK BLOB(1 M) INLINE LENGTH 1024
  ) ORGANIZE BY COLUMN;

[Back to Top](#top)
<a id='quotes'></a>

## QUOTE Indentifier and Quote Literal

The `QUOTE_IDENT` function returns a string that can be used as an identifier in an SQL statement. The input value is a string expression which will have the proper quotes placed around it so that it can be used as a valid name in a SQL statement. The `QUOTE_LITERAL` function returns a string that can be used as a string in an SQL statement. The input value is a string expression which will have the proper quotes placed around it so that it can be used as a valid literal in a SQL statement.

These functions can be used when constructing dynamic SQL statements, or for generating values for the new JSON functions. Db2 (the SQL language) requires special handling of quotes which is different from many programming languages.

A Db2 column name, *without* special characters, does not require any quotes surrounding it. The standard always folds the string into uppercase, so consider the following SQL.

In [ ]:
%%sql 
DROP TABLE DEMO;
CREATE TABLE DEMO
  (
  firstname varchar(10),
  lastname  varchar(10)
  );
DESCRIBE TABLE DEMO;

Describing the table structure will show that the column names are in uppercase.

In [ ]:
%sql DESCRIBE TABLE DEMO

If we wanted to use lowercase column names then we will need to delimit them with double quotes as shown in the following SQL.

In [ ]:
%%sql 
DROP TABLE DEMO;
CREATE TABLE DEMO
  (
  "firstname" varchar(10),
  "lastname"  varchar(10)
  );
DESCRIBE TABLE DEMO;

Describing the table structure will now show that the column names are in lowercase.

In [ ]:
%sql DESCRIBE TABLE DEMO

When generating column names we can surround the input strings with the `QUOTE_IDENT` function to make sure that the quotes are properly placed around the names. Note how uppercase identifiers that **do not** have special characters do not require quotes to be placed around them.

In [ ]:
%%sql -grid
VALUES ('HELLO WORLD',  QUOTE_IDENT('HELLO WORLD'))
UNION ALL 
VALUES ('HELLOWORLD',   QUOTE_IDENT('HELLOWORLD'))
UNION ALL
VALUES ('HELLO_WORLD',  QUOTE_IDENT('HELLO_WORLD'))
UNION ALL
VALUES ('hello world',  QUOTE_IDENT('hello world'))
UNION ALL
VALUES ('hello"world',  QUOTE_IDENT('hello"world'))
UNION ALL
VALUES ('hello''world', QUOTE_IDENT('hello''world'))
UNION ALL
VALUES ('',             QUOTE_IDENT(''))

The `QUOTE_LITERAL` function makes sure that a string has the proper quotes placed inside the string. Db2 requires that single quotes within a string be escaped with two quote characters. So the following is a valid Db2 string:
```
'This is a single quote '' within a Db2 string'
```

All Db2 strings must start and end with the single quote character `'`, so any imbedded quotes must be duplicated (i.e. you need two of them `''`) to represent one quote character. 

This first example shows how an integer value will be quoted.

In [ ]:
%sql VALUES QUOTE_LITERAL(42.5)

When dealing with application variables, you don't always know if the quotes are properly delimited. This example has multiple quote characters in the string.

In [ ]:
quotes = "O'Brian went to O'Hare airport for a flight to Hawai'i"
results = %sql -r VALUES QUOTE_LITERAL(:quotes)
results[1][0]

[Back to Top](#top)
<a id='oracle'></a>

## Oracle Compatibility Libraries

Db2 continues to update the Oracle compatibility modules that are available in the database. In order to use these libraries, you must turn on the Oracle compatibility vector **before creating a database**. You do not need to have complete Oracle compatibility on to use these functions. The following Db2 setting should be turned on to get these compatibility libraries to work. 
```sql
DB2_COMPATIBILITY_VECTOR=400
```

**Note:** If the database you are connecting to does not have this compatibility vector set, then none of the functions below will work.

The libraries that are new in this release are:
* `DBMS_APPLICATION_INFO`

  Includes procedures that set custom client info exported through some of the table functions in the database and helps identify the targeted sessions upon executing the procedure.
  

* `DBMS_LOCK` (Currently in beta)   
  
  Provides lock management functionality that allows SQL PL developers to control concurrent access to critical resources in their applications.
  

* `UTL_RAW`

  Provides a set of routines for manipulating binary data of the data type VARBINARY. These routines perform various functions including data conversion, casting, comparison, concatenation, substring, xrange and translation.
  
The schema for all procedures and functions in these modules is `SYSIBMADM`

### DBMS_APPLICATION_INFO
The `DBMS_APPLICATION_INFO` library is used for tracking and debugging routines that are running in the database. The functions that are found in this library inlcudes:

* `READ_CLIENT_INFO ( client_info )`

  Reads and returns the value of the client information field from the current session
 
 
* `READ_MODULE ( module_name, action_name )`

  Reads and returns the value of the module and actions fields from the current session


* `SET_CLIENT_INFO ( client_info )`

  Sets and registers the value of the client information field from the current session


* `SET_MODULE ( module_name, action_name )`

  Sets and registers the module name that is currently being executed


* `SET_ACTION ( action_name )`

  Sets and registers the action name within the current module


* `SET_SESSION_LONGOPS( rindex, slno, op_name, target, context, sofar, totalwork, target_desc, units )`

  Sets and registers a row in the SYSTOOLS.SESSION_LONGOPS table, to store progress information for long operations


The following call will place information into a catalog table about the procedure called `add_employee`.

In [ ]:
%sql CALL DBMS_APPLICATION_INFO.SET_MODULE('customer_maintenance','update address'); 

To retrieve this information you would use the `READ_MODULE` function.

In [ ]:
%sql CALL DBMS_APPLICATION_INFO.READ_MODULE( ?, ? )

To reset the information we set the values to null.

In [ ]:
%sql CALL DBMS_APPLICATION_INFO.SET_MODULE('null','null'); 

### UTL_RAW
The `UTL_RAW` library provides a set of routines for manipulating binary data of the data type `VARBINARY`. These routines perform various functions including data conversion, casting, comparison, concatenation, substring, xrange and translation.The functions that are found in this library inlcudes:

* `BIT_AND ( x,y )` − bitwise logical AND operation against x, y
* `BIT_OR ( x,y )` − bitwise logical OR operation against x, y
* `BIT_XOR ( x,y )` − bitwise logical EXCLUSIVE OR operation against x,y
* `BIT_COMPLEMENT ( x )` − bitwise logical COMPLEMENT operation against x
* `COMPARE ( x, y, pad )` − compares two values with optional padding 
* `CONCAT ( x1, x2, …, x12 )` − concatenates up to twelve (12) VARBINARY values into a single value
* `COPIES ( x, n )` − returns the concatenated results of the VARBINARY value a specified number times
* `LENGTH ( x )` − returns the length of a VARBINARY value
* `REVERSE ( x )` − reverses the order of digits of a VARBINARY value
* `SUBSTR ( x, start, length )` − returns a specified portion of a VARBINARY value
* `CAST_TO_RAW` − casts a VARCHAR value to a VARBINARY value
* `CAST_TO_VARCHAR2` − casts a VARBINARY value to a VARCHAR2 value
* `CAST_FROM_NUMBER` − casts a DECFLOAT value to a VARBINARY value
* `CAST_TO_NUMBER` − casts a VARBINARY value to a DECFLOAT value
* `CAST_FROM_BINARY_DOUBLE` − casts a DOUBLE value to a VARBINARY value
* `CAST_FROM_BINARY_FLOAT` − casts a FLOAT value to a VARBINARY value
* `CAST_FROM_BINARY_INTEGER` − casts an INTEGER value to a VARBINARY value
* `CAST_TO_BINARY_DOUBLE` − casts a VARBINARY value to a DOUBLE value
* `CAST_FROM_BINARY_FLOAT` − casts a VARBINARY value to a FLOAT value
* `CAST_FROM_BINARY_INTEGER` − casts a VARBINARY value to an INTEGER value


**AND** performs a bitwise logical AND operation against x, y

In [ ]:
import binascii
result = %sql -r VALUES UTL_RAW.BIT_AND( bx'0D', bx'0B')
print(binascii.hexlify(result[1][0]))

**OR** performs a bitwise logical OR operation against x, y

In [ ]:
result = %sql -r VALUES UTL_RAW.BIT_OR( bx'0D', bx'0B' )
print(binascii.hexlify(result[1][0]))

**XOR** performs a bitwise logical EXCLUSIVE OR operation against x,y

In [ ]:
result = %sql -r VALUES UTL_RAW.BIT_XOR( bx'0D', bx'0B' )
print(binascii.hexlify(result[1][0]))

**COMPLEMENT** performs a bitwise logical COMPLEMENT operation against x

In [ ]:
result = %sql -r VALUES UTL_RAW.BIT_COMPLEMENT( bx'0D')
print(binascii.hexlify(result[1][0]))

**COMPARE** compares two values with optional padding. If the two values are equal, the result is zero. If the values differ, the returned value is the byte of the first value that differs from the second. 

In [ ]:
%sql VALUES UTL_RAW.COMPARE ( bx'010D', bx'010C' )

**CONCAT** concatenates up to twelve (12) VARBINARY values into a single value.

In [ ]:
result = %sql -r VALUES UTL_RAW.CONCAT ( bx'010D', bx'010C' )
print(binascii.hexlify(result[1][0]))

**COPIES** returns the concatenated results of the VARBINARY value a specified number times.

In [ ]:
result = %sql -r VALUES UTL_RAW.COPIES ( bx'010D', 3 )
print(binascii.hexlify(result[1][0]))

**LENGTH** returns the length of a VARBINARY value in bytes.

In [ ]:
%sql VALUES UTL_RAW.LENGTH ( bx'010D010CFFFF')

**REVERSE** reverses the order of digits of a VARBINARY value. Note that this doesn't flip the bits in the byte values.

In [ ]:
result = %sql -r VALUES UTL_RAW.REVERSE ( bx'010D010CFFFF')
print(binascii.hexlify(result[1][0]))

**SUBSTR** returns a specified portion of a VARBINARY value. Byte positions start at 1. 

In [ ]:
result = %sql -r VALUES UTL_RAW.SUBSTR( bx'010D010CFFFF', 2, 2)
print(binascii.hexlify(result[1][0]))

[Back to Top](#top)
<a id='function'></a>

#### Credits: IBM 2019, George Baklarz [baklarz@ca.ibm.com]